In [6]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('Data/weatherAUS.csv')
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.40,22.90,0.60,nan,nan,W,44.00,W,...,22.00,1007.70,1007.10,8.00,nan,16.90,21.80,No,0.00,No
1,2008-12-02,Albury,7.40,25.10,0.00,nan,nan,WNW,44.00,NNW,...,25.00,1010.60,1007.80,nan,nan,17.20,24.30,No,0.00,No
2,2008-12-03,Albury,12.90,25.70,0.00,nan,nan,WSW,46.00,W,...,30.00,1007.60,1008.70,nan,2.00,21.00,23.20,No,0.00,No
3,2008-12-04,Albury,9.20,28.00,0.00,nan,nan,NE,24.00,SE,...,16.00,1017.60,1012.80,nan,nan,18.10,26.50,No,1.00,No
4,2008-12-05,Albury,17.50,32.30,1.00,nan,nan,W,41.00,ENE,...,33.00,1010.80,1006.00,7.00,8.00,17.80,29.70,No,0.20,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.50,21.80,0.00,nan,nan,E,31.00,ESE,...,27.00,1024.70,1021.20,nan,nan,9.40,20.90,No,0.00,No
142189,2017-06-21,Uluru,2.80,23.40,0.00,nan,nan,E,31.00,SE,...,24.00,1024.60,1020.30,nan,nan,10.10,22.40,No,0.00,No
142190,2017-06-22,Uluru,3.60,25.30,0.00,nan,nan,NNW,22.00,SE,...,21.00,1023.50,1019.10,nan,nan,10.90,24.50,No,0.00,No
142191,2017-06-23,Uluru,5.40,26.90,0.00,nan,nan,N,37.00,SE,...,24.00,1021.00,1016.80,nan,nan,12.50,26.10,No,0.00,No
